<a href="https://colab.research.google.com/github/YongERong/stackup-sentimental-chatbot/blob/main/Llama_Chatbot_with_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup
Installation and importing of packages, inserting of API keys/tokens

In [ ]:
!pip install -q accelerate protobuf sentencepiece torch transformers huggingface_hub gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 53.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 65.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 9.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import os
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from huggingface_hub import login
import torch
import gradio as gr

## Chat with Context

In [ ]:
from pprint import pprint
from transformers import Pipeline, AutoTokenizer, AutoModelForCausalLM
import torch

# chat buffer
buffer = []

# function to embed messages in template format
def embed_message(message, role):
    return {
        "role": role,
        "content": message
    }

# custom pipeline
class ChatBufferPipeline(Pipeline):

    def _sanitize_parameters(self, **kwargs):
        preprocess_kwargs = {}

        # lookback on chat
        if "lookback" in kwargs:
            preprocess_kwargs["lookback"] = kwargs["lookback"]

        if "sentiment" in kwargs:
            preprocess_kwargs["sentiment"] = kwargs["sentiment"]

        return preprocess_kwargs, {}, {}

    def preprocess(self, prompt, lookback=None, sentiment=None):
        # initial system message
        messages = [
            {
                "role": "system",
                "content": "You are a friendly chatbot who answers user questions. You can use the previous examples if this helps you. The sentiment score of the question is also provided.",
                "sentiment": sentiment if sentiment else None
            },
        ]
        # get chat history

        if lookback:
            buffer_messages = buffer[-(lookback):]
            messages += buffer_messages
        # embed user message in template format
        user_message = embed_message(prompt, "user")
        messages.append(user_message)
        # add new message to buffer
        buffer.append(user_message)

        messages = self.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        return self.tokenizer(messages, return_tensors="pt").input_ids.cuda()

    def _forward(self, model_inputs):
        outputs = self.model.generate(model_inputs, max_new_tokens=250, min_new_tokens=20)
        return {"outputs": outputs, "inputs": model_inputs}

    def postprocess(self, model_outputs):
        outputs = model_outputs["outputs"]
        inputs = model_outputs["inputs"]
        assistant_output = self.tokenizer.decode(outputs[0][len(inputs[0]):], add_special_tokens=False)
        buffer.append(embed_message(assistant_output, "assistant"))
        full_dialog = self.tokenizer.decode(outputs[0])
        return assistant_output, full_dialog

In [ ]:
# model
model_id = "HuggingFaceH4/zephyr-7b-beta"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", torch_dtype=torch.float16)

# pipeline
chatpipe = ChatBufferPipeline(model=model, tokenizer=tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

...

model-00006-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00007-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

model-00008-of-00008.safetensors:   0%|          | 0.00/816M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

## Sentiment Analysis
Attempted to use the same model with a different prompt -> switch to distilBERT for faster speeds

### distilBERT

In [ ]:
sentiment_pipe = pipeline("text-classification", model="distilbert/distilbert-base-uncased-finetuned-sst-2-english")

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


## Gradio UI

In [ ]:
def gradio_chat_interface(question):
    sentiment = sentiment_pipe(question)
    response, logs = chatpipe("My favorite color is yellow, what is yours?", lookback=10, sentiment=sentiment)
    return f"Answer from Assistant {response}\n\n {logs}"

In [ ]:
# Create a Gradio Interface
interface = gr.Interface(
    fn=gradio_chat_interface,
    inputs="text",
    outputs="text",
    title="Zephyr Chatbot with Sentiment Analysis",
    description="Ask a question and the chatbot will respond using a Zephyr Model",
)

In [ ]:
# Launch the Gradio Interface
interface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://0b5fe99e343e5a6f17.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
